In [15]:
import pandas as pd
df1 = pd.read_csv('PA_metadata_train-1.csv')
df2 = pd.read_csv('PA_metadata_train-2.csv')
df = pd.concat([df1, df2], ignore_index=True)

In [16]:
print(df.head())

        lon        lat    year  geoUncertaintyInM  areaInM2         region  \
0  3.099038  43.134956  2021.0                5.0     100.0  MEDITERRANEAN   
1  3.099038  43.134956  2021.0                5.0     100.0  MEDITERRANEAN   
2  3.099038  43.134956  2021.0                5.0     100.0  MEDITERRANEAN   
3  3.099038  43.134956  2021.0                5.0     100.0  MEDITERRANEAN   
4  3.099038  43.134956  2021.0                5.0     100.0  MEDITERRANEAN   

  country  speciesId  surveyId                county  ... 55.33177  2019  \
0  France     6874.0     212.0  Languedoc-Roussillon  ...      NaN   NaN   
1  France      476.0     212.0  Languedoc-Roussillon  ...      NaN   NaN   
2  France    11157.0     212.0  Languedoc-Roussillon  ...      NaN   NaN   
3  France     8784.0     212.0  Languedoc-Roussillon  ...      NaN   NaN   
4  France     4530.0     212.0  Languedoc-Roussillon  ...      NaN   NaN   

   10.0  79.0  CONTINENTAL  Denmark 7661 1976273  Zealand  Naestved Kommun

In [3]:
print(df.columns)

Index(['lon', 'lat', 'year', 'geoUncertaintyInM', 'areaInM2', 'region',
       'country', 'speciesId', 'surveyId', 'county', 'district'],
      dtype='object')


In [4]:
df = df[['lon', 'lat', 'year', 'speciesId']]

df = df.dropna()

print(df.head())

        lon        lat  year  speciesId
0  3.099038  43.134956  2021       6874
1  3.099038  43.134956  2021        476
2  3.099038  43.134956  2021      11157
3  3.099038  43.134956  2021       8784
4  3.099038  43.134956  2021       4530


In [5]:
species_6874_rows = df[df['speciesId'] == 6874]

print(species_6874_rows)


               lon        lat  year  speciesId
0         3.099038  43.134956  2021       6874
2780      4.177610  43.848390  2017       6874
3112     15.485710  42.110800  2020       6874
3554      2.339040  43.145010  2018       6874
4185      5.486416  43.204942  2019       6874
...            ...        ...   ...        ...
1471906   3.144531  43.194955  2021       6874
1471946   9.374250  40.938296  2018       6874
1476908   5.672178  43.214294  2021       6874
1478750   2.878387  43.047919  2021       6874
1479381   9.277770  42.613390  2017       6874

[924 rows x 4 columns]


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Create a GeoDataFrame from the dataset
geometry = gpd.points_from_xy(df['lon'], df['lat'])
gdf = gpd.GeoDataFrame(df, geometry=geometry)

# Set up the plot
fig, ax = plt.subplots(figsize=(25, 20))

# Plot all points with a single marker
gdf.plot(ax=ax, marker='o', color='blue', markersize=5)  # Use 'o' marker in blue color

# Add a grid to the plot
ax.grid(True, linestyle='--', alpha=0.5)  # Customize grid style and transparency

# Customize the plot
plt.title("Tree Distribution")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

In [13]:
from sklearn.metrics import silhouette_score

# Take a random sample of your DataFrame if it's too large
sample_size = 1000  # Adjust as needed
df_sampled = df.sample(n=sample_size, random_state=1)  # Use a random seed for reproducibility

# Compute silhouette score on the sampled data
if -1 in df_sampled['cluster'].values:
    valid_clusters = df_sampled[df_sampled['cluster'] != -1]
else:
    valid_clusters = df_sampled

silhouette_avg = silhouette_score(valid_clusters[['lon', 'lat']], valid_clusters['cluster'])
print(f"Silhouette Score: {silhouette_avg}")

Silhouette Score: 0.15255222915197517


In [ ]:
import matplotlib.pyplot as plt

# Set up the plot
fig, ax = plt.subplots(figsize=(25, 20))

# Plot the cluster centroids
ax.scatter(average_coordinates['lon'], average_coordinates['lat'], 
           c='red', marker='X', s=10, label='Cluster Centroids')  # 'X' marker for centroids

# Customize the plot
plt.title("Cluster Centroids of Tree Data")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.legend(loc="upper left")
plt.grid(True, linestyle='--', alpha=0.5)  # Add a grid for reference
plt.show()

In [ ]:
# import geopandas as gpd
# import matplotlib.pyplot as plt
# from sklearn.cluster import DBSCAN
# from scipy.spatial import ConvexHull
# import numpy as np

# # Assuming df is your DataFrame containing 'lon', 'lat', and 'speciesId'
# # Prepare the data for clustering
# coordinates = df[['lon', 'lat']].values

# # Apply DBSCAN
# dbscan = DBSCAN(eps=0.005, min_samples=5)  # Adjust eps based on your data density
# df['cluster'] = dbscan.fit_predict(coordinates)

# # Create a GeoDataFrame from the dataset
# geometry = gpd.points_from_xy(df['lon'], df['lat'])
# gdf = gpd.GeoDataFrame(df, geometry=geometry)

# # Set up the plot
# fig, ax = plt.subplots(figsize=(15, 10))

# # Plot all points with a single marker
# gdf.plot(ax=ax, marker='o', color='blue', markersize=5, label='Trees')  # Use 'o' marker in blue color

# # Plot cluster boundaries
# for cluster_id in gdf['cluster'].unique():
#     if cluster_id != -1:  # Skip noise points
#         cluster_points = gdf[gdf['cluster'] == cluster_id]
#         if len(cluster_points) >= 3:  # Need at least three points to form a convex hull
#             hull = ConvexHull(cluster_points[['lon', 'lat']].values)
#             hull_points = np.append(hull.vertices, hull.vertices[0])  # Close the polygon
#             ax.plot(cluster_points.iloc[hull_points]['lon'], 
#                     cluster_points.iloc[hull_points]['lat'], 
#                     color='red', alpha=0.5)  # Draw the boundary in red

# # Add a grid to the plot
# ax.grid(True, linestyle='--', alpha=0.5)  # Customize grid style and transparency

# # Customize the plot
# plt.title("Tree Distribution with Cluster Boundaries")
# plt.xlabel("Longitude")
# plt.ylabel("Latitude")
# plt.legend()
# plt.show()

In [ ]:
import requests

# OpenWeatherMap API credentials
api_key = "00e8e808c9ee63e1a6848d838daabe6d"

# Function to get all air pollution components from OpenWeatherMap for given lat and lon
def get_air_pollution_data(lat, lon):
        url = f"http://api.openweathermap.org/data/2.5/air_pollution?lat={lat}&lon={lon}&appid={api_key}"
        
        # Make the API request
        response = requests.get(url)
        
        if response.status_code == 200:
            data = response.json()  # Parse the response to JSON
            main_aqi = data['list'][0]['main']['aqi']  # AQI value
            
            components = data['list'][0]['components']  # All pollutant components
            co2 = components.get('co2', None)
            co = components.get('co', None)       # Carbon Monoxide
            no = components.get('no', None)       # Nitric Oxide
            no2 = components.get('no2', None)     # Nitrogen Dioxide
            o3 = components.get('o3', None)       # Ozone
            so2 = components.get('so2', None)     # Sulfur Dioxide
            pm25 = components.get('pm2_5', None)  # Fine Particulate Matter (PM2.5)
            pm10 = components.get('pm10', None)   # Coarse Particulate Matter (PM10)
            nh3 = components.get('nh3', None)     # Ammonia

            return main_aqi, co2, co, no, no2, o3, so2, pm25, pm10, nh3
        
        else:
            return [None] * 9  # Return None for all fields if the request fails

    # Applying the function to each row in the dataset
    df[['AQI', 'CO2', 'CO', 'NO', 'NO2', 'O3', 'SO2', 'PM2.5', 'PM10', 'NH3']] = pd.DataFrame(
        df.apply(lambda row: get_air_pollution_data(row['latitude_coordinate'], row['longitude_coordinate']), axis=1).tolist()
    )

    # units of all data is µg/m³

In [ ]:
# Save the updated DataFrame to a new CSV file
df.to_csv('dataset_with_aqi.csv', index=False)

print("AQI data added successfully!")